In [1]:
import pandas as pd
from elements import Player, Lobby, SkillBasedMatchMaking
from itertools import permutations, combinations
from dataclasses import asdict

In [2]:
df = pd.read_csv('./player_data.csv')
df['index'] = list(range(len(df)))
print(df.shape)

(300, 3)


In [3]:
df.sample(10)

,Player,KD,index
218,P219,3.745856,218
4,P5,1.595561,4
122,P123,2.021105,122
40,P41,2.013263,40
69,P70,2.592140,69
175,P176,2.041747,175
41,P42,2.844860,41
130,P131,1.828485,130
32,P33,2.969554,32
125,P126,2.428684,125


In [4]:
players = [Player(index=p['index'], kill_score=p['KD']) for p in df.to_dict('records')]
lobbies = [Lobby(index=0,name='A', score=4.25),
           Lobby(index=1,name='B', score=5.0),
           Lobby(index=2,name='C', score=5.75)]

In [5]:
index_to_player = {rec['index']: rec for rec in df.to_dict('records')}
index_to_lobby = {loby.index : asdict(loby) for loby in lobbies}

In [6]:
skillBasedMatching = SkillBasedMatchMaking(players=players, lobbies=lobbies, party_index=[(0, 21), (1, 33)])

In [7]:
%%time

skillBasedMatching.process()

CPU times: user 1min 1s, sys: 4.1 s, total: 1min 5s
Wall time: 1min 8s


In [8]:
skillBasedMatching.status

0

In [9]:
l_f =[]
s_v = []
for k,v in skillBasedMatching.X.items():
    if v.solution_value() > 0.1:
        l_f.append(k)
        s_v.append(v.solution_value())

In [10]:
final_op = []
for _ in l_f:
    p1, p2, l = index_to_player[_[0]], index_to_player[_[1]], index_to_lobby[_[2]]
    final_op.append({'player_1': p1['Player'],
                     'player_1_kill_score': p1['KD'],
                     'player_2': p2['Player'],
                     'player_2_kill_score': p2['KD'],
                     'total_score': (p1['KD'] + p2['KD']),
                     'lobby': l['name'],
                     'lobby_score': l['score']})

In [11]:
df_final = pd.DataFrame(final_op)

In [12]:
df_final['lobby'].value_counts()

A    104
B    100
C     96
Name: lobby, dtype: int64

In [13]:
df_final.sample(10)

,player_1,player_1_kill_score,player_2,player_2_kill_score,total_score,lobby,lobby_score
14,P15,2.157291,P210,2.846651,5.003942,B,5.00
96,P97,2.399041,P70,2.592140,4.991181,B,5.00
116,P117,1.464216,P294,2.787445,4.251661,A,4.25
231,P232,1.806293,P66,2.440846,4.247138,A,4.25
174,P175,2.718417,P251,3.024878,5.743295,C,5.75
180,P181,2.806838,P191,1.436427,4.243264,A,4.25
31,P32,2.816044,P54,1.427121,4.243164,A,4.25
226,P227,1.445262,P134,2.805601,4.250864,A,4.25
97,P98,1.475676,P49,3.531070,5.006746,B,5.00
286,P287,2.995428,P172,2.754931,5.750360,C,5.75


In [14]:
df_final.to_csv("team_results.csv", index=False)